# Example of usage of an adaptative optimizer

## Quantum Autoencoders

In this notebook, we will present an example with quantum autoencoders. Here, we are given a set of states $\{\psi_i\}_{i=1}^N$ and we want to compress such data by reducing the number of qubits needed to represent it. Given the regiser of qubits with $n_A + n_B$ qubits, one trains a gate sequence $U(\theta)$ to compress the ensemble of states to $n_A$ qubits, $n_B$ qubits acting as trash. 

The task is to optimize the following cost:
$\sum_i  p_i Tr_{AB}[H_{L} U(\theta) |\psi_i \rangle \langle \psi_i| U(\theta)^\dagger]$


where $H_{L} = \mathbb{1}_{AB} - \frac{1}{n_B} \sum_{j=1}^{n_B} \mathbb{1}_{A} \otimes |0 \rangle \langle 0|_j \mathbb{1}_{\bar{j}} $

In our example, $\{\rho_i\}_{i=1}^N$ will be taken from the collection of datasets available in Pennylane. We take several states obtained by running VQE for different bond lengths of the H2 molecule in the STO-3G basis. We obtain a dataset of 42 circuits as follows:

In [1]:
import pennylane as qml
from pennylane import numpy as np
from refoqus import Refoqus

bondlengths = ['0.5', '0.54', '0.58', '0.62', '0.66', '0.7', '0.74', '0.742', '0.78', '0.82', '0.86', '0.9', '0.94', '0.98', '1.02', '1.06', '1.1', '1.14', '1.18', '1.22', '1.26', '1.3', '1.34', '1.38', '1.42', '1.46', '1.5', '1.54', '1.58', '1.62', '1.66', '1.7', '1.74', '1.78', '1.82', '1.86', '1.9', '1.94', '1.98', '2.02', '2.06', '2.1']
moldataset = qml.data.load("qchem", molname="H2", basis="STO-3G", bondlength=bondlengths)
nbdatapoints = len(moldataset)

Now we set the coefficients as $- \frac{1}{n_B}$ for the individual terms and of the hamiltonian of interest $H_L$ (note $1$ is a constant to be added later).

In [5]:
nbqbits = len(moldataset[0].hamiltonian.wires)
nbtrash = nbqbits // 2
coefficients_cost = [- 1.0 / float(nbtrash)] * nbtrash

projector = np.zeros((2, 2))
projector[0, 0] = 1


quantoencoder_hamiltonian_term = [qml.Hermitian(projector,wires=i) for i in range(nbtrash)]

hermitian_of_interest = qml.Hamiltonian(coefficients_cost, quantoencoder_hamiltonian_term)

Next, we define functions to evaluate the true cost during optimization.

In [9]:
analytic_dev = qml.device("default.qubit", wires=nbqbits, shots=None)

@qml.qnode(analytic_dev)
def cost_analytic_one_circuit(weights, index_datapoint):
    
    for op in moldataset[index_datapoint].vqe_gates:
        qml.apply(op)
        
    StronglyEntanglingLayers(weights, wires=analytic_dev.wires)
    return qml.expval(hermitian_of_interest)

def cost_analytic_alldataset(weights):
    
    cost = 0.0
    for m in range(nbdatapoints):
        cost += cost_analytic_one_circuit(weights, m)
    cost = 1.0 + cost / nbdatapoints
    return cost

Now, the ansatz is defined as with StronglyEntanglingLayers. We also sample initial values and the corresponding cost.

In [10]:
from pennylane.templates.layers import StronglyEntanglingLayers

# hyperparameter of ansatz
num_layers = 3


param_shape = StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=nbqbits)
init_params = np.random.uniform(low=0.0, high=2*np.pi, size=param_shape, requires_grad=True)
cost_analytic_alldataset(init_params)

tensor(0.40522901, requires_grad=True)

Our adaptative optimizer will be Refoqus where we provide the necessary arguments as follows and we perform niter iterations.

In [11]:
opt = Refoqus(moldataset, quantoencoder_hamiltonian_term, coefficients_cost, param_shape, min_shots=2)
params = init_params
niter = 20

cost_refoqus = [cost_analytic_alldataset(params)]
shots_refoqus = [0]

for i in range(niter):
    params = opt.step(params)
    cost_refoqus.append(cost_analytic_alldataset(params))
    shots_refoqus.append(opt.shots_used)
    print(f"Step {i}: cost = {cost_refoqus[-1]}, shots_used = {shots_refoqus[-1]}")

Step 0: cost = 0.39200306394205686, shots_used = 144
Step 1: cost = 0.4033870587298968, shots_used = 478
Step 2: cost = 0.2330648863477156, shots_used = 854
Step 3: cost = 0.24636451603891185, shots_used = 1454
Step 4: cost = 0.23474490078718835, shots_used = 2060
Step 5: cost = 0.1772365214542122, shots_used = 3270
Step 6: cost = 0.15126990632560522, shots_used = 4986
Step 7: cost = 0.12985101035690794, shots_used = 7876
Step 8: cost = 0.11006448853276818, shots_used = 13196
Step 9: cost = 0.08512065113266643, shots_used = 20064
Step 10: cost = 0.07897058426603343, shots_used = 27094
Step 11: cost = 0.05989148046284076, shots_used = 45868
Step 12: cost = 0.043150713022650056, shots_used = 57220
Step 13: cost = 0.031541715510777335, shots_used = 67258
Step 14: cost = 0.023043393729625272, shots_used = 77540
Step 15: cost = 0.01789347764399718, shots_used = 88022
Step 16: cost = 0.014951192264402668, shots_used = 101774
Step 17: cost = 0.011469031947357378, shots_used = 124514
Step 18: 